In [20]:
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler
import mlflow

In [92]:
df = pd.read_csv(r'C:\Users\Acer\Documents\bitcoin_price_prediction\data\btc_raw.csv')
print(f"Loaded {len(df)} rows")

Loaded 2102 rows


In [94]:
df = df[['Open','Close']].dropna()
df = df.drop(df.index[0])
print(df, len(df))

                   Open              Close
2      3746.71337890625   3843.52001953125
3      3849.21630859375  3943.409423828125
4     3931.048583984375    3836.7412109375
5       3832.0400390625  3857.717529296875
6     3851.973876953125  3845.194580078125
...                 ...                ...
2097       63138.546875     65181.01953125
2098      65180.6640625      65790.6640625
2099      65792.1796875      65887.6484375
2100      65888.8984375      65635.3046875
2101        65634.65625            63329.5

[2100 rows x 2 columns] 2100


In [68]:
close_series = df['Close'].copy()
close_series = close_series.dropna()
close_series

0                 BTC-USD
2        3843.52001953125
3       3943.409423828125
4         3836.7412109375
5       3857.717529296875
              ...        
2097       65181.01953125
2098        65790.6640625
2099        65887.6484375
2100        65635.3046875
2101              63329.5
Name: Close, Length: 2101, dtype: object

In [ ]:
close_series[]

'31676.693359375'

In [51]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(close_series.values.reshape(-1, 1))


In [52]:
dv_path_test = "dv.b"

In [53]:
with open(dv_path_test, "wb") as f_in:
    pickle.dump(scaler, f_in)

In [54]:
pd.DataFrame(
        X_scaled,
        index=close_series.index,
        columns=['scaled_close']
    ).to_csv(r"C:\Users\Acer\Documents\bitcoin_price_prediction\data\X_processed_test.csv")

In [38]:
import xgboost

In [55]:
import pandas as pd
import xgboost as xgb
import mlflow
import mlflow.xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [60]:
X = pd.read_csv(r"C:\Users\Acer\Documents\bitcoin_price_prediction\data\X_processed_test.csv").values

In [61]:
X

array([[ 2.00000000e+00, -1.30735301e+00],
       [ 3.00000000e+00, -1.30220821e+00],
       [ 4.00000000e+00, -1.30770215e+00],
       ...,
       [ 2.09900000e+03,  1.88822445e+00],
       [ 2.10000000e+03,  1.87522751e+00],
       [ 2.10100000e+03,  1.75646724e+00]], shape=(2100, 2))

In [62]:
y = X[:, 0]

In [63]:
y

array([2.000e+00, 3.000e+00, 4.000e+00, ..., 2.099e+03, 2.100e+03,
       2.101e+03], shape=(2100,))

In [71]:
# Use only Open and Close
data = df[['Open', 'Close']].dropna()

In [74]:
data = data.drop(0)
data

,Open,Close
2,3746.71337890625,3843.52001953125
3,3849.21630859375,3943.409423828125
4,3931.048583984375,3836.7412109375
5,3832.0400390625,3857.717529296875
6,3851.973876953125,3845.194580078125
...,...,...
2097,63138.546875,65181.01953125
2098,65180.6640625,65790.6640625
2099,65792.1796875,65887.6484375
2100,65888.8984375,65635.3046875


In [76]:
# Features and target
X_raw = data[['Open']].values
y_raw = data['Close'].values

In [77]:
# Fit scalers
scaler_X = StandardScaler()
scaler_y = StandardScaler()

In [78]:
X_scaled = scaler_X.fit_transform(X_raw)
y_scaled = scaler_y.fit_transform(y_raw.reshape(-1, 1)).flatten()

In [79]:
# Save both scalers in one pickle (your "DV")
dv = {
        'scaler_X': scaler_X,
        'scaler_y': scaler_y
    }
with open("dv.b", "wb") as f:
    pickle.dump(dv, f)

In [80]:
pd.DataFrame({
        'scaled_open': X_scaled.flatten(),
        'scaled_close': y_scaled
    }, index=data.index).to_csv(r"C:\Users\Acer\Documents\bitcoin_price_prediction\data\processed_test.csv")

In [81]:
import pandas as pd
import xgboost as xgb
import mlflow
import mlflow.xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np

In [87]:
df = pd.read_csv(r"C:\Users\Acer\Documents\bitcoin_price_prediction\data\processed_test.csv", index_col=0, parse_dates=True)
X = df[['scaled_open']].values
y = df['scaled_close'].values

C:\Users\Acer\AppData\Local\Temp\ipykernel_11592\2278999769.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(r"C:\Users\Acer\Documents\bitcoin_price_prediction\data\processed_test.csv", index_col=0, parse_dates=True)


In [89]:
X.shape
y.shape

(2100,)

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [91]:

model = xgb.XGBRegressor(n_estimators=100, max_depth=3, random_state=42)
model.fit(X_train, y_train)
    
    # Predict (in scaled space)
y_pred_scaled = model.predict(X_test)
mae_scaled = mean_absolute_error(y_test, y_pred_scaled)

In [97]:
model_name="btc_open_to_close"
stage = None
run_id = "cb4be5245e244161bbe9bb45aa01f3f4"

model = mlflow.pyfunc.load_model(f"models:/{model_name}/{stage}")

MlflowException: Registered Model with name=btc_open_to_close not found